<a href="https://colab.research.google.com/github/jacqueline-chung/AIBusiness/blob/master/ADDITIONAL_DON'T_RUN_ITM_703_Course_Project_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries and Load TensorBoard**

In [ ]:
# Import libraries for building neural networks
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

# Import for TensorBoard
from tensorboard.plugins.hparams import api as hp
from tensorboard import summary as summary_lib
from keras.callbacks import TensorBoard

# Import for ROC and AUC
from sklearn.metrics import roc_auc_score

# Mount your google drive to your Google Colab
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Import for metrics
from keras import backend as K

# Import for Data Exploration
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Get the id of your file by right click on it on your google drive 
# Link: https://drive.google.com/open?id=1KuReb5jz_mu0qbzQsJMjjHLMzU2E67lC
downloaded2 = drive.CreateFile({'id':'1KuReb5jz_mu0qbzQsJMjjHLMzU2E67lC'}) 
downloaded2.GetContentFile('heart.csv')  

downloaded = drive.CreateFile({'id':'1KuReb5jz_mu0qbzQsJMjjHLMzU2E67lC'}) 
downloaded.GetContentFile('cd.csv')  

# Use Tensor 2.0
debug = True
_verbose=1 if debug else 0

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load inline
%matplotlib inline

# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# **Split Training and Test Data**


*  Split the data randomly, where training data is 70% and test data is 30%
* Print the data that was split between training and test

Please refer to the followin link for more infomation about using loc function,
https://www.w3resource.com/pandas/series/series-loc.php
on this link you can find a section to use a bolean values to select the rows


In [ ]:
# The following code generates a list with same length as our data, and randomly assigns true or false to them 
split = 0.7
# Compare with split because if loading msk, you will get T or F -> subset dataframes
msk = np.random.rand(len(data)) < split

# Shuffle the entire data set (applies to both train & test)
#  Sample method return a random sample of data , and by setting fraction to 1, we'll get the whole data in a shuffled way
data = data.sample(frac=1).reset_index(drop=True)

# Using panadas loc function to subset our list
# Training Data
train_labels = data.loc[msk, data.columns =='target']
train_data  = data.loc[msk, data.columns !='target']
# Testing Data
test_labels = data.loc[~msk, data.columns =='target']
test_data  = data.loc[~msk, data.columns !='target']

In [ ]:
# Look at first rows of data to check training and test data
print('Test Data:', test_data.head(1))
print('Test Target Class:', test_labels.head(1))
print('Training Data:', train_data.head(1))
print('Training Target Class:', train_labels.head(1))

 # Random numbers that are generated (based on # of rows in dataset)
print(msk[0:5])
# Check shape of each dataset
print(train_data.shape)
print(test_data.shape)
# Check the number of values in total
print(test_data.shape[0]+train_data.shape[0])

Test Data:    age  sex  cp  trestbps  chol  fbs  ...  thalach  exang  oldpeak  slope  ca  thal
1   52    1   1       134   201    0  ...      158      0      0.8      2   1     2

[1 rows x 13 columns]
Test Target Class:    target
1       1
Training Data:    age  sex  cp  trestbps  chol  fbs  ...  thalach  exang  oldpeak  slope  ca  thal
0   48    1   1       110   229    0  ...      168      0      1.0      0   0     3

[1 rows x 13 columns]
Training Target Class:    target
0       0
[ True False False  True  True]
(197, 13)
(106, 13)
303


### Cross Entropy

(Code doesn't work)

In [ ]:
# Import libraries for Cross-Entropy
from sklearn.model_selection import LeaveOneOut,KFold
from sklearn.model_selection import StratifiedKFold

# Declare features for data
feature_names = data.columns
X = data.drop(data.columns[-1], axis=1)
y = data['target']

skf = StratifiedKFold(n_splits=5, random_state=None)
# X is the feature set and y is the target
for train_index, test_index in skf.split(X,y): 
    print("Train:", train_index, "Validation:", test_index) 
    train_data, test_data = X[train_index], X[test_index] 
    train_labels, test_labels = y[train_index], y[test_index]

# **Normalization**
The numerical values stay between [RANGE]

In [ ]:
# Normalizing our data: setting the axis to 0 means that the mean and std 
#  should calculated towards rows not columns
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)
train_data = (train_data - mean) / std
test_data = (test_data - mean) / std

In [ ]:
# Check training and test data after normalization
print(train_data.head(1))
print(test_data.head(1))

        age       sex        cp  ...     slope        ca      thal
0 -0.681558  0.740529  0.069829  ... -2.287915 -0.679177  1.175925

[1 rows x 13 columns]
      age       sex        cp  trestbps  ...   oldpeak     slope        ca      thal
1 -0.2377  0.740529  0.069829  0.154293  ... -0.184308  0.920051  0.342181 -0.455464

[1 rows x 13 columns]


# **Define Hyperparameters**

### 1 Hidden Layer

In [ ]:
'''
# We create the parameters that we would like to try different values for.
# we use the HParam method to parametrize our model so we investigate the effect of different parameters. 

# Adjust # of neurons in input (first) layer
HP_INPUT = hp.HParam('input', hp.Discrete([4,16,24]))

# Adjust # of neurons in the hidden (second) layer 
HP_HIDDEN1 = hp.HParam('hidden 1', hp.Discrete([4,8])) 

# Use different optimizer to see which one gives better accuracy
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

# Use the most common activation functions
HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu','sigmoid']))

# Use 5 epochs for now
HP_EPOCH = hp.HParam('epoch', hp.Discrete([5]))

# Use common loss function
HP_LOSS = hp.HParam('loss', hp.Discrete(['sparse_categorical_crossentropy']))

# Our goal is to find the values for the above paramters that gives the best accuracy on the test data
ACCURACY_METRIC = 'accuracy'
'''

"\n# We create the parameters that we would like to try different values for.\n# we use the HParam method to parametrize our model so we investigate the effect of different parameters. \n\n# Adjust # of neurons in input (first) layer\nHP_INPUT = hp.HParam('input', hp.Discrete([4,16,24]))\n\n# Adjust # of neurons in the hidden (second) layer \nHP_HIDDEN1 = hp.HParam('hidden 1', hp.Discrete([4,8])) \n\n# Use different optimizer to see which one gives better accuracy\nHP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))\n\n# Use the most common activation functions\nHP_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu','sigmoid']))\n\n# Use 5 epochs for now\nHP_EPOCH = hp.HParam('epoch', hp.Discrete([5]))\n\n# Use common loss function\nHP_LOSS = hp.HParam('loss', hp.Discrete(['sparse_categorical_crossentropy']))\n\n# Our goal is to find the values for the above paramters that gives the best accuracy on the test data\nACCURACY_METRIC = 'accuracy'\n"

### 2 Hidden Layers

In [ ]:
# This shows us the best hyperparameters for the neural network classifiers

# We create the parameters that we would like to try different values for.
# We use the HParam method to parametrize our model so we investigate the effect of different parameters. 

# Adjust # of neurons in input (first) layer
HP_INPUT = hp.HParam('input', hp.Discrete([500,750,1000]))

# Adjust # of neurons in the first hidden (second) layer 
HP_HIDDEN1  = hp.HParam('hidden 1', hp.Discrete([8,16,50]))

# Adjust # of neurons in the second hidden (third) layer 
HP_HIDDEN2 = hp.HParam('hidden 2', hp.Discrete([8,16,50]))

# Use different optimizer to see which one gives better accuracy
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam','sgd']))

# Use Activation function called Tanh and ReLU
HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['tanh', 'relu']))

# Use 5 and 10 epochs
HP_EPOCH = hp.HParam('epoch', hp.Discrete([5,10]))

# Use common loss function: Binary Cross-Entropy, Hinge, Squared Hinge
HP_LOSS = hp.HParam('loss', hp.Discrete(['binary_crossentropy','hinge','squared_hinge']))

# Declare 'accuracy' variable
ACC_METRIC = 'accuracy'

# Our goal is to find the values for the above paramters that gives the best performance on the test data

### 3 Hidden Layers

In [ ]:
# This shows us the best hyperparameters for the neural network classifiers

# We create the parameters that we would like to try different values for.
# We use the HParam method to parametrize our model so we investigate the effect of different parameters. 

# Adjust # of neurons in input (first) layer
HP_INPUT = hp.HParam('input', hp.Discrete([500,750,1000]))

# Adjust # of neurons in the first hidden (second) layer 
HP_HIDDEN1  = hp.HParam('hidden 1', hp.Discrete([8,16,50]))

# Adjust # of neurons in the second hidden (third) layer 
HP_HIDDEN2 = hp.HParam('hidden 2', hp.Discrete([8,16,50]))

# Adjust # of neurons in the third hidden (fourth) layer 
HP_HIDDEN2 = hp.HParam('hidden 3', hp.Discrete([8,16,50]))

# Use different optimizer to see which one gives better accuracy
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam','sgd']))

# Use Activation function called Tanh and ReLU
HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['tanh', 'relu']))

# Use 5 and 10 epochs
HP_EPOCH = hp.HParam('epoch', hp.Discrete([5,10]))

# Use common loss function: Binary Cross-Entropy, Hinge, Squared Hinge
HP_LOSS = hp.HParam('loss', hp.Discrete(['binary_crossentropy','hinge','squared_hinge']))

# Declare 'accuracy' variable
ACC_METRIC = 'accuracy'

# Our goal is to find the values for the above paramters that gives the best performance on the test data

# **Define Metrics**
Create functions for Recall, Precision, F1 Score to measure performance for models.

The 3 following metrics, in addition to classification accuracy, that are commonly required for a neural network model on a binary classification problem are:

* Precision
* Recall
* F1 Score

We will calculate these three metrics, as well as classification accuracy using the scikit-learn metrics API, as well as loss.

Links:
* https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/

In [ ]:
# Define recall_m using training and test labels 
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# Define precision_m using training and test labels 
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# Define f1_m using training and test labels 
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# **Parameter Sensitivity Analysis**
Change Hyper Parameters for TensorBoard
This will overide the initial hyperparameters
Due to the limitations of Google Colab, we broke up the code in different files to run each iteration, so these hyperparameters were actually done separately, but it was what we tested to get our final results.

* no of epochs
* no of neurons (first)
* no of neurons (hidden)
* activation function
* learning rate (if you have time)
* no of layers

Try adding slowly, so the program doesn't crash

No of neurons (first layer)

In [ ]:
# Adjust # of neurons in input (first) layer 
HP_INPUT = hp.HParam('input', hp.Discrete([3,4,8,9,12,16,18,25,30,50,75,100,150,250,500,750,1000]))

No of Neurons (Hidden Layer)
* This is advance: The number can be decided using formulas (i.e. 75% of input should be used for units for hidden (but not fixed since it's based on probaility))

In [ ]:
# Adjust # of neurons in hidden (second) layer (smaller increments)
HP_HIDDEN1 = hp.HParam('hidden 1', hp.Discrete([3,4,8,9,12,16,18,25,30,50,75,100,150,250,500,750,1000])) 

No of Epochs

Depending on the optimizer and activation functions, the # of epochs may improve or increase overfitting. However, it seems that having more epochs is good to an extent. Having too little can make it more difficult for the model to learn. We have to find the sweet spot.

In [ ]:
HP_EPOCH = hp.HParam('epoch', hp.Discrete([5, 10, 30, 50, 75, 100, 200]))

Activation Functions 

We're starting with the most common ones, then we will work our way to other functions:
*  Sigmoid 
*  ReLU
* Softmax (use less - not as common)
* Tanh

Links:
* https://keras.io/activations/
* https://towardsdatascience.com/hyper-parameters-in-action-a524bf5bf1c
* https://www.tensorflow.org/api_docs/python/tf/keras/activations?version=nightly

Note: Not all the activation functions will work due to the shape of the data.


In [ ]:
HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu', 'sigmoid', 'softmax', 'tanh']))

Multiple Activation Functions

In [ ]:
# Change the activation functions for each layer
HP_ACTIVATION1 = hp.HParam('activation (input)', hp.Discrete(['relu', 'tanh','sigmoid']))

HP_ACTIVATION2 = hp.HParam('activation (hidden 1)', hp.Discrete(['relu', 'tanh','sigmoid']))

HP_ACTIVATION3 = hp.HParam('activation (hidden 2)', hp.Discrete(['relu', 'tanh','sigmoid']))

HP_ACTIVATION4 = hp.HParam('activation (hidden 3)', hp.Discrete(['relu', 'tanh','sigmoid']))

HP_ACTIVATION4 = hp.HParam('activation (hidden 3)', hp.Discrete(['relu', 'tanh','sigmoid']))

HP_ACTIVATION5 = hp.HParam('activation (output)', hp.Discrete(['relu', 'tanh','sigmoid']))

In [ ]:
# The following changes would be made to the structure:

# Build model with hyperparameters and activation functions
def build_model(hparams):
  model = keras.Sequential([
    keras.layers.Dense(hparams[HP_INPUT], hparams[HP_ACTIVATION1],  # put our parameter (replace parameter with # of parameter)
                       input_shape=(train_data.shape[1],)), # Input Layer
    keras.layers.Dense(hparams[HP_HIDDEN1], hparams[HP_ACTIVATION2]), # Hidden Layer
    #keras.layers.Dense(hparams[HP_HIDDEN2], hparams[HP_ACTIVATION]), # Hidden Layer
    #keras.layers.Dense(hparams[HP_HIDDEN3], hparams[HP_ACTIVATION]), # Hidden Layer
    keras.layers.Dense(2, hparams[HP_ACTIVATION3]) # Output Layer
])
  
  # Compiling model; Calculate Accuracy; Optimizer optimizes loss function 
  model.compile(hparams[HP_OPTIMIZER],hparams[HP_LOSS],ACCURACY_METRIC)
  #model.compile([HP_OPTIMIZER](hparams[HP_LR]),hparams[HP_LOSS],ACCURACY_METRIC)

# Fit and train model and save metrics in history
  model.fit(train_data, train_labels, hparams[HP_EPOCH], 
            validation_split = 0.2, verbose = 1)
  
  _, accuracy = model.evaluate(test_data, test_labels)
  return accuracy # Output model

# With statement
with tf.summary.create_file_writer('./hparam_tuning').as_default():
  hp.hparams_config(
      hparams=[HP_INPUT,HP_HIDDEN1,HP_OPTIMIZER, HP_ACTIVATION1, HP_ACTIVATION2, HP_ACTIVATION3,HP_ACTIVATION4, HP_ACTIVATION5,  HP_EPOCH, HP_LOSS],
      metrics=[hp.Metric(ACCURACY_METRIC, display_name='Accuracy')]
  )

# This function will execute the model builing method with different set of paratemer and log the results to the folder.
# With statement helps you to keep your log file open while executing the commands in the with block
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # Record the values used in this trial
    loss, recall, precision, f1, accuracy = build_model(hparams)
    tf.summary.scalar('loss', loss, step=1)
    tf.summary.scalar('recall', recall, step=1) # Scalar: a data class; different value
    tf.summary.scalar('precision', precision, step=1) # Scalar: a data class; different value
    tf.summary.scalar('f1', f1, step=1)
    tf.summary.scalar(ACC_METRIC, accuracy, step=1)

# Run our model by using hyperparameters that defined earlier
# Combined all of them in for loops to run the different combinations of parameters
session_num = 0
for input1 in HP_INPUT.domain.values:
  for hidden1 in HP_HIDDEN1.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        for activation1 in HP_ACTIVATION1.domain.values:
                for activation2 in HP_ACTIVATION2.domain.values:
                    for activation3 in HP_ACTIVATION3.domain.values:
                        for activation4 in HP_ACTIVATION4.domain.values:
                            for activation5 in HP_ACTIVATION5.domain.values:
                                for epoch in HP_EPOCH.domain.values:
                                    for loss in HP_LOSS.domain.values:
                                            hparams = {
                                                HP_INPUT: input1,
                                                HP_HIDDEN1: hidden1,
                                                HP_OPTIMIZER: optimizer,
                                                HP_ACTIVATION1: activation1,
                                                HP_ACTIVATION2: activation2,
                                                HP_ACTIVATION3: activation3, 
                                                HP_ACTIVATION4: activation4, 
                                                HP_ACTIVATION5: activation5,                                    
                                    }
                                    run_name = "run-%d" % session_num 
                                    print('--- Starting trial: %s' % run_name) 
                                    print({h.name: hparams[h] for h in hparams}) # this runs the parameters
                                    run('./hparam_tuning/' + run_name, hparams)
                                    session_num += 1

Learning rate - more advanced

Higher learnerning rate will decrease performance and increase overfitting, while lower learning rate will have less reliance on previous instances. This will help decrease overfitting. However, it is possible to underfit, it would make the model too simple and it would need more epochs (and vice versa).
This is why we need the if statements to run, instead of the hyperparameters. 

Links:
* https://machinelearningmastery.com/learning-rate-for-deep-learning-neural-networks/
* https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/scalars_and_keras.ipynb

(The code didn't work)

In [ ]:
HP_LR = hp.HParam('learning rate', hp.Discrete([0.1,0.5]))

In [ ]:
def lr_schedule(epoch):
  """
  Returns a custom learning rate that decreases as epochs progress.

  Should I use hyperparams??
  """
  learning_rate = 0.2
  if epoch > 10:
    learning_rate = 0.02
  if epoch > 20:
    learning_rate = 0.01
  if epoch > 50:
    learning_rate = 0.005

  tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
  return learning_rate

lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)
tensorboard_callback = keras.callbacks.TensorBoard()

Loss - if you have time

In [ ]:
HP_LOSS = hp.HParam('loss', hp.Discrete(['sparse_categorical_crossentropy', 'binary_crossentropy','hinge','squared_hinge','categorical_crossentropy','kullback_leibler_divergence']))

No of Hidden Layers


*   1
*   2
* 3

From research, it says, on average, 2 hidden layers are best. It depends how complicated the data is, but since this data is pretty small, we will stick with 2 to 3. Having more hidden layers may lead to overfitting.


No of Neurons (Hidden Layer 2)

In [ ]:
# Adjust # of neurons in hidden (second) layer (smaller increments)
HP_HIDDEN2 = hp.HParam('hidden 2', hp.Discrete([3,4,8,9,12,16,18,25,30,50,75,100,150,250,500,750,1000])) 

No of Neurons (Hidden Layer 3)

In [ ]:
# Adjust # of neurons in hidden (second) layer (smaller increments)
HP_HIDDEN3 = hp.HParam('hidden 3', hp.Discrete([3,4,8,9,12,16,18,25,30,50,75,100,150,250,500,750,1000])) 

# **Neural Network Classifier**


### 2 Hidden Layers

In [ ]:
# Build model with hyperparameters and activation functions
def build_model(hparams):
  model = keras.Sequential([
    keras.layers.Dense(hparams[HP_INPUT], hparams[HP_ACTIVATION],  # put our parameter (replace parameter with # of parameter)
                       input_shape=(train_data.shape[1],)), # Input Layer
    keras.layers.Dense(hparams[HP_HIDDEN1], hparams[HP_ACTIVATION]), # Hidden Layer
    keras.layers.Dense(hparams[HP_HIDDEN2], hparams[HP_ACTIVATION]), # Hidden Layer
    #keras.layers.Dense(hparams[HP_HIDDEN3], hparams[HP_ACTIVATION]), # Hidden Layer
    keras.layers.Dense(2, hparams[HP_ACTIVATION]) # Output Layer
])
  
  # Compiling model by calculating accuracy, recall, precision, and F1 score
  # Use Optimizer optimizes loss function 
  model.compile(hparams[HP_OPTIMIZER],hparams[HP_LOSS],metrics=[recall_m, precision_m, f1_m, ACC_METRIC])

# Fit and train model and save metrics in history
  model.fit(train_data, train_labels, hparams[HP_EPOCH], 
            validation_split = 0.2, verbose = 1)
  
  # Evaluate the model using test data
  loss = model.evaluate(test_data, test_labels)
  _,recall, precision, f1, accuracy = model.evaluate(test_data, test_labels)

  # Get loss, recall, precision, F1 score, and accuracy after evaluating model against test data
  return loss[0], recall, precision, f1, accuracy # Output model

### 3 Hidden Layers

In [ ]:
## Build model with hyperparameters and activation functions
def build_model(hparams):
  model = keras.Sequential([
    keras.layers.Dense(hparams[HP_INPUT], hparams[HP_ACTIVATION],  # put our parameter (replace parameter with # of parameter)
                       input_shape=(train_data.shape[1],)), # Input Layer
    keras.layers.Dense(hparams[HP_HIDDEN1], hparams[HP_ACTIVATION]), # Hidden Layer
    keras.layers.Dense(hparams[HP_HIDDEN2], hparams[HP_ACTIVATION]), # Hidden Layer
    keras.layers.Dense(hparams[HP_HIDDEN3], hparams[HP_ACTIVATION]), # Hidden Layer
    keras.layers.Dense(2, hparams[HP_ACTIVATION]) # Output Layer
])
  
  # Compiling model; Calculate Accuracy; Optimizer optimizes loss function 
  model.compile(hparams[HP_OPTIMIZER],hparams[HP_LOSS],metrics=[recall_m, precision_m, f1_m, ACC_METRIC])
  #model.compile([HP_OPTIMIZER](hparams[HP_LR]),hparams[HP_LOSS],ACCURACY_METRIC)

# Fit and train model and save metrics in history
  model.fit(train_data, train_labels, hparams[HP_EPOCH], 
            validation_split = 0.2, verbose = 1)
  
  loss = model.evaluate(test_data, test_labels)
  _,recall, precision, f1, accuracy = model.evaluate(test_data, test_labels)

  return loss[0], recall, precision, f1, accuracy # Output model

Using TensorFlow backend.


# **Add Metrics**
* Accuracy 
* Running Time (Epoch)
* Precision
* Recall
* F1 Score
* Loss

In [ ]:
# With statement by adding the hyperparameters in the summary
with tf.summary.create_file_writer('./hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_INPUT,HP_HIDDEN1, HP_HIDDEN2,  HP_OPTIMIZER, HP_ACTIVATION, HP_EPOCH, HP_LOSS],
    metrics=[hp.Metric('loss', display_name='loss'),
             hp.Metric('recall', display_name='recall'),
             hp.Metric('precision', display_name='precision'),
             hp.Metric('f1', display_name='f1_score'),
             hp.Metric(ACC_METRIC, display_name='accuracy')]
  )

## **Run Models with Hyparam**

## 2 Hidden Layers

In [ ]:
# This function will execute the model builing method with different set of paratemer and log the results to the folder.
# With statement helps you to keep your log file open while executing the commands in the with block
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # Record the values used in this trial
    loss, recall, precision, f1, accuracy = build_model(hparams)
    tf.summary.scalar('loss', loss, step=1)
    tf.summary.scalar('recall', recall, step=1) # Scalar: a data class; different value
    tf.summary.scalar('precision', precision, step=1) # Scalar: a data class; different value
    tf.summary.scalar('f1', f1, step=1)
    tf.summary.scalar(ACC_METRIC, accuracy, step=1)

In [ ]:
# Run our model by using hyperparameters that defined earlier
# Combined all of them in for loops to run the different combinations of parameters
session_num = 0

for input1 in HP_INPUT.domain.values:
  for hidden1 in HP_HIDDEN1.domain.values:
      for hidden2 in HP_HIDDEN2.domain.values:
            for optimizer in HP_OPTIMIZER.domain.values:
                for activation in HP_ACTIVATION.domain.values:
                    for epoch in HP_EPOCH.domain.values:
                        for loss in HP_LOSS.domain.values:         
                                hparams = {
                                    HP_INPUT: input1,
                                    HP_HIDDEN1: hidden1,
                                    HP_OPTIMIZER: optimizer,
                                    HP_ACTIVATION: activation,
                                    HP_EPOCH: epoch,
                                    HP_LOSS: loss,
                                    HP_HIDDEN2: hidden2,                                   
                                }
                                # Run the hyperparameters 
                                run_name = "run-%d" % session_num 
                                print('--- Starting trial: %s' % run_name) 
                                print({h.name: hparams[h] for h in hparams}) 
                                run('./hparam_tuning/' + run_name, hparams)
                                session_num += 1 # Add session_num

## 3 Hidden Layers

In [ ]:
 # too technical right now
# With statement
with tf.summary.create_file_writer('./hparam_tuning').as_default():
  hp.hparams_config(
      #hparams=[HP_INPUT,HP_HIDDEN1,HP_OPTIMIZER, HP_ACTIVATION, HP_EPOCH, HP_LOSS],
    hparams=[HP_INPUT,HP_HIDDEN1, HP_HIDDEN2, HP_HIDDEN3,  HP_OPTIMIZER, HP_ACTIVATION, HP_EPOCH, HP_LOSS],
    metrics=[hp.Metric('loss', display_name='loss'),
             hp.Metric('recall', display_name='recall'),
             hp.Metric('precision', display_name='precision'),
             hp.Metric('f1', display_name='f1_score'),
             hp.Metric(ACC_METRIC, display_name='accuracy')]
  )

# This function will execute the model builing method with different set of paratemer and log the results to the folder.
# With statement helps you to keep your log file open while executing the commands in the with block
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # Record the values used in this trial
    loss, recall, precision, f1, accuracy = build_model(hparams)
    tf.summary.scalar('loss', loss, step=1)
    tf.summary.scalar('recall', recall, step=1) # Scalar: a data class; different value
    tf.summary.scalar('precision', precision, step=1) # Scalar: a data class; different value
    tf.summary.scalar('f1', f1, step=1)
    tf.summary.scalar(ACC_METRIC, accuracy, step=1)

# Run our model by using hyperparameters that defined earlier
# Combined all of them in for loops to run the different combinations of parameters
session_num = 0
for input1 in HP_INPUT.domain.values:
  for hidden1 in HP_HIDDEN1.domain.values:
      for hidden2 in HP_HIDDEN2.domain.values:
          for hidden3 in HP_HIDDEN3.domain.values:
            for optimizer in HP_OPTIMIZER.domain.values:
                for activation in HP_ACTIVATION.domain.values:
                    for epoch in HP_EPOCH.domain.values:
                        for loss in HP_LOSS.domain.values:
          
                                hparams = {
                                    HP_INPUT: input1,
                                    HP_HIDDEN1: hidden1,
                                    HP_OPTIMIZER: optimizer,
                                    HP_ACTIVATION: activation,
                                    HP_EPOCH: epoch,
                                    HP_LOSS: loss,
                                    HP_HIDDEN2: hidden2,
                                    HP_HIDDEN3: hidden3                                    
                                }
                                run_name = "run-%d" % session_num 
                                print('--- Starting trial: %s' % run_name) 
                                print({h.name: hparams[h] for h in hparams}) # this runs the parameters
                                run('./hparam_tuning/' + run_name, hparams)
                                session_num += 1

# **Testing & Experiemental Results**
Use TensorBoard


In [ ]:
# Loads an inplace TensorBoard to investigate the performance of  runs
# This can sort performance
# It allows you to compare the various hyperparameters
%tensorboard --logdir ./hparam_tuning